In [1]:
from oauth import oauth
import nltk
import string
import os
import regex as re
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem.porter import PorterStemmer
from credentials import twitter_oauth
#http://brandonrose.org/clustering

In [2]:
import twitter as t
#from oauth import 


#-----------------------------------------------------------------------
# create twitter API object
#-----------------------------------------------------------------------
twitter = t.Api(twitter_oauth[0],twitter_oauth[1],twitter_oauth[2],twitter_oauth[3])


#-----------------------------------------------------------------------
# perform a basic search 
# Twitter API docs:
# https://dev.twitter.com/rest/reference/get/search/tweets
#-----------------------------------------------------------------------
query = twitter.GetSearch(raw_query="q=%20publix&result_type=recent&since=2017-06-23&count=1000")

for i in range(1,100):
    maxid = min([q.id for q in query])
    qq = twitter.GetSearch(raw_query="q=%20publix&result_type=recent&since=2017-06-01&count=100&max_id="+str(maxid))
    query = query +qq 


#-----------------------------------------------------------------------
# How long did this query take?
#-----------------------------------------------------------------------
#print "Search complete (%.3f seconds)" % (query["search_metadata"]["completed_in"])

print (len(query))

10000


In [3]:
print(query[99].id)
print(query[0].id)


881159329650532352
881182419210129408


In [4]:
nltk.download('punkt')
token_dict = {}
stemmer = PorterStemmer()

# here I define a tokenizer and stemmer which returns the set of stems in the text that it is passed

def tokenize_and_stem(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    stems = [stemmer.stem(t) for t in filtered_tokens]
    return stems


def tokenize_only(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word.lower() for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    return filtered_tokens

def stem_tokens(tokens, stemmer):
    stemmed = []
    for item in tokens:
        stemmed.append(stemmer.stem(item))
    return stemmed

def tokenize(text):
    tokens = nltk.word_tokenize(text)
    stems = stem_tokens(tokens, stemmer)
    return stems

for q in query:
    text = q.text
    lowers = text.lower()
    no_punctuation = re.sub(ur"\p{P}+", "", lowers)
#    print no_punctuation, q.id,q.hashtags,q.favorite_count,q.favorited,q.retweet_count,q.user.screen_name,q.user_mentions
    token_dict[q.id] = no_punctuation

tweets = [t.text for t in query]
#this can take some time
tfidf = TfidfVectorizer(tokenizer=tokenize_and_stem, stop_words='english')
tfs = tfidf.fit_transform(tweets)

[nltk_data] Downloading package punkt to /Users/malarcon/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [5]:


#not super pythonic, no, not at all.
#use extend so it's a big flat list of vocab
totalvocab_stemmed = []
totalvocab_tokenized = []
for i in tweets:
    allwords_stemmed = tokenize_and_stem(i) #for each item in 'synopses', tokenize/stem
    totalvocab_stemmed.extend(allwords_stemmed) #extend the 'totalvocab_stemmed' list
    
    allwords_tokenized = tokenize_only(i)
    totalvocab_tokenized.extend(allwords_tokenized)

vocab_frame = pd.DataFrame({'words': totalvocab_tokenized}, index = totalvocab_stemmed)
#print 'there are ' + str(vocab_frame.shape[0]) + ' items in vocab_frame'

In [6]:
vocab_frame

,words
Im,im
take,taking
zfreakingb,zfreakingb
to,to
publix,publix
first,first
and,and
foremost,foremost
then,then
he,he


In [7]:
tfidf.get_feature_names()

[u"'cotton",
 u"'cube",
 u"'d",
 u"'em",
 u"'good",
 u"'ing",
 u"'ll",
 u"'m",
 u"'merica",
 u"'n",
 u"'not",
 u"'pillori",
 u"'pyu",
 u"'re",
 u"'s",
 u"'safeti",
 u"'should",
 u"'teet",
 u"'til",
 u"'u",
 u"'v",
 u"'ve",
 u"'well",
 u'***no',
 u'**new**',
 u'**sneak',
 u'*blurt',
 u'*c',
 u'*cricket*',
 u'*expired*',
 u'*goe',
 u'*in',
 u'*lol*',
 u'*my*',
 u'*nsync',
 u'*off-duty*',
 u'*play',
 u'*publix',
 u'*really*',
 u'*run',
 u'*so*',
 u'*sob*',
 u'*take',
 u'*tripl',
 u'*update*',
 u'*walk',
 u'*whispers*',
 u'+hr+',
 u'-8ball',
 u'-a',
 u'-call',
 u'-chickfila',
 u'-gecko',
 u'-ishbel',
 u'-junk',
 u'-kroger',
 u'-larg',
 u'-mi',
 u'-notic',
 u'-peopl',
 u'-publix',
 u'-purpl',
 u'-realli',
 u'-sedimentari',
 u'-sincer',
 u'-sweet',
 u'-the',
 u'-walmart',
 u'..and',
 u'..can',
 u'..i',
 u'..it',
 u'..now',
 u'..on',
 u'..thank',
 u'..they',
 u'..with',
 u'.all',
 u'.and',
 u'.bought',
 u'.but',
 u'.chicken',
 u'.do',
 u'.especi',
 u'.fight',
 u'.go',
 u'.hard',
 u'.have',
 u

In [8]:
from sklearn.metrics.pairwise import cosine_similarity
dist = 1 - cosine_similarity(tfs)
print
print

In [18]:
from sklearn.cluster import KMeans

num_clusters = 10

km = KMeans(n_clusters=num_clusters)

%time km.fit(tfs)

clusters = km.labels_.tolist()


CPU times: user 17 s, sys: 275 ms, total: 17.3 s
Wall time: 9.95 s


In [19]:
clusters = km.labels_.tolist()
clusters

[9,
 9,
 9,
 0,
 9,
 9,
 0,
 9,
 0,
 9,
 2,
 5,
 0,
 0,
 9,
 0,
 3,
 9,
 9,
 9,
 9,
 9,
 9,
 3,
 9,
 8,
 9,
 9,
 9,
 9,
 5,
 2,
 3,
 9,
 9,
 3,
 9,
 9,
 9,
 2,
 9,
 9,
 9,
 4,
 3,
 9,
 3,
 9,
 3,
 4,
 9,
 9,
 0,
 5,
 9,
 9,
 9,
 9,
 2,
 9,
 0,
 5,
 3,
 3,
 9,
 9,
 3,
 9,
 8,
 9,
 8,
 8,
 5,
 5,
 9,
 9,
 2,
 9,
 3,
 3,
 9,
 9,
 9,
 9,
 3,
 9,
 0,
 9,
 3,
 9,
 3,
 3,
 9,
 3,
 9,
 5,
 9,
 9,
 9,
 9,
 9,
 9,
 3,
 3,
 3,
 3,
 9,
 9,
 5,
 3,
 3,
 9,
 3,
 3,
 9,
 3,
 8,
 5,
 9,
 5,
 5,
 8,
 3,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 5,
 9,
 8,
 8,
 9,
 9,
 9,
 9,
 9,
 9,
 8,
 9,
 9,
 2,
 9,
 9,
 9,
 9,
 9,
 5,
 9,
 9,
 7,
 9,
 5,
 5,
 5,
 9,
 9,
 5,
 2,
 9,
 9,
 2,
 9,
 9,
 9,
 3,
 9,
 9,
 9,
 9,
 0,
 0,
 9,
 9,
 9,
 9,
 9,
 9,
 3,
 9,
 9,
 9,
 9,
 3,
 0,
 9,
 9,
 9,
 0,
 0,
 9,
 0,
 0,
 0,
 9,
 9,
 0,
 9,
 9,
 9,
 5,
 8,
 9,
 9,
 3,
 9,
 8,
 9,
 5,
 9,
 9,
 9,
 5,
 2,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 8,
 3,
 9,
 4,
 4,
 5,
 9,
 5,
 9,
 9,
 5,
 9,
 5,
 9,
 9,
 5,
 2,


In [20]:
tweets = { 'text': [q.text for q in query], 'id': [q.id for q in query], 
          'retweet_count':q.retweet_count, 'hashtags': [q.hashtags for q in query], 
          'favorited': [q.favorited for q in query], 'screen_name': [q.user.screen_name for q in query], 
          'user_mentions': [q.user_mentions for q in query],'cluster':clusters }
frame = pd.DataFrame(tweets, index = [clusters] )

In [21]:
frame['cluster'].value_counts() #number of films per cluster (clusters from 0 to 4)


9    6313
5     800
2     740
8     581
4     574
1     295
0     255
3     204
6     122
7     116
Name: cluster, dtype: int64

In [22]:
grouped = frame['retweet_count'].groupby(frame['cluster']) #groupby retweet_count for aggregation purposes

grouped.mean() #average retweet_count per cluster

cluster
0    0
1    0
2    0
3    0
4    0
5    0
6    0
7    0
8    0
9    0
Name: retweet_count, dtype: int64

In [23]:
terms = tfidf.get_feature_names()

from __future__ import print_function

print("Top terms per cluster:")
print()
#sort cluster centers by proximity to centroid
order_centroids = km.cluster_centers_.argsort()[:, ::-1] 

for i in range(num_clusters):
    print("Cluster %d words:" % i, end='')
    
    for ind in order_centroids[i, :6]: #replace 6 with n words per cluster
        print(' %s' % vocab_frame.ix[terms[ind].split(' ')].values.tolist()[0][0]#.encode('utf-8', 'ignore')
              , end=',')
    print() #add whitespace
    print() #add whitespace
    
    print("Cluster %d tweets:" % i, end='')
    for title in frame.ix[i]['text'].values.tolist():
        print('\n\t* %s' % title)
    print() #add whitespace
    print() #add whitespace
    
print()
print()

Top terms per cluster:

Cluster 0 words: thanks, nan, shopping, nan, publix, sharing,

Cluster 0 tweets:
	* @jasminebrooks44 I can look into this for you! ^ML

	* @AnthonyEspo93 I will pass along your suggestion! ^ML

	* @Pete_Wingerr Enjoy your trip! ^ML

	* @BckhdGodfather Our pleasure! ^ML

	* @galexgong Thanks for the kind words! ^ML

	* @stevenjr27 So adorable! Thanks for sharing this pic with us! ^ML

	* Thank you Publix. National Enquirer is the only magazine covered. https://t.co/OJHqguHtHw

	* @JasonDennisWTVM Still a nice couple too! Thanks for saying hi today at Publix.

	* @Publix Thank you for the awesome snack while shopping! https://t.co/mxd6dPCMSc

	* @KLevine15 Enjoy! ^ML

	* @pmallory21 Thanks! I will share your suggestion! ^ML

	* @naysha_elish You already know. ^ML

	* @iBe_Vibin Thanks fam.  ^ML

	* @pmallory21 Do you have a zip code in mind? ^ML

	* @Abaybeee We love you too! ^ML

	* @Abaybeee We love you too! ^ML

	* @DTPfromATL What store did you visit? ^ML

	* 

In [24]:
#-----------------------------------------------------------------------
# Loop through each of the results, and print its content.
#-----------------------------------------------------------------------
for q in query:
    print (q.text,q.hashtags,q.favorite_count,q.favorited,q.retweet_count,q.user.screen_name,q.user_mentions)


Im taking @zfreakingb to publix first and foremost, then he can meet my family. The scent of publix subs on him will calm them [] 0 False 0 SudKnee [User(ID=761226391866122240, ScreenName=zfreakingb)]
RT @DeKalbNewsNow: Armed carjacking reported at Publix in DeKalb:
https://t.co/SsLegRGssX [] 0 False 1 Bmdot [User(ID=84135492, ScreenName=DeKalbNewsNow)]
@kaitlyndcoleman publix shopper [] 0 False 0 benji_cam [User(ID=857042174, ScreenName=kaitlyndcoleman)]
@jasminebrooks44 I can look into this for you! ^ML [] 0 False 0 Publix [User(ID=700738628259962880, ScreenName=jasminebrooks44)]
@DannyAller @alloutofforks @RunningNoleFan @Publix But beef ribs though [] 1 False 0 Jmacfsu24 [User(ID=65819089, ScreenName=DannyAller), User(ID=817070150297985024, ScreenName=alloutofforks), User(ID=2713372164, ScreenName=RunningNoleFan), User(ID=14784550, ScreenName=Publix)]
I went grocery shopping at publix today and only spent 80$. What a time to be alive [] 0 False 0 __lostprincess []
@AnthonyEspo93 I 

ValueError: I/O operation on closed file

In [25]:
q = query[0]#['Status']
#print q.text
#print q.favorited
q.id

881182419210129408

In [26]:
len(query)

10000